# Load and initialize

settings

In [ ]:
# sample number
j='All'
n_samples = 8000
    # create visualization
visualize = True
    # show figures
show = True

load_from_file = False

envirement

In [ ]:
import os
import time 
start = time.process_time()
import pathlib
import sys
from IPython.display import Image, display
from tqdm import tqdm_notebook,tqdm
# import time

import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context

plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"
%matplotlib inline

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180) #according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

# from scipy import signal, stats
# import seaborn as sns

# from sklearn import metrics

# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

# from lmfit import minimize, Parameters


parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir+'/functions/')


from usefull_functions import *
from functions import *
from plot_functions import *
from clustering_functions import *



%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%autoreload 2


dir_data = parent_dir+'/Data/'
# 'Data/'
dir_plots = parent_dir+'/Plots_sample'+j+'/'
saveSVG  =False
settings =        (dir_plots,show,saveSVG)
folderExists(dir_plots)

TF_ENABLE_ONEDNN_OPTS=0

load data; normalized (CD45-)  

In [ ]:
if load_from_file:
    k_orig = pickle_load('k'+j+'_sample',dir_data )['k']
else:
    
    # dict = pickle_load('k'+j+'_dict',dir_data )
    k_orig =pickle_load('k'+j+'_dict',dir_data )['k']
    # downsample
    k_orig= subsample_k(k_orig.copy(),n=n_samples)

print(len(k_orig))
# if j=='2' or j=='1':
#     kCD45pos = subsample_k(pickle_load('k'+j+'_dict',dir_data )['pos'],n=2000) 
#     kCD45pos['Clust'] =-2
#     print('kCD45pos2 loaded')


dict = pickle_load('k'+j+'_names_dict',dir_data )
NamesAll = dict['NamesAll']
CellIden = dict['CellIden']
EpiCols = dict['EpiCols']
Core = dict['Core']


CellIden_EpiCols = CellIden + EpiCols
groups = [['CellIden','EpiCols','CellIden+EpiCols'],[CellIden,EpiCols,CellIden_EpiCols]]
del dict


Visualize data

no point to print data.describe() as it is whitened (std~1, mean~0) 

# Cell Identity

In [ ]:
k = k_orig.copy()
CAll=pd.concat([k]).copy()
s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}


name = 'Tumor'+j+'_CellIden_'


### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['1']=0.07
s['n_neighbors']['1']=200
# 0.05,150
s['min_dist']['2']=0.001
s['n_neighbors']['2']=60

s['min_dist']['4']=0.001
s['n_neighbors']['4']=60

s['min_dist']['5']=0.01
s['n_neighbors']['5']=10

s['min_dist']['All']=0.01
s['n_neighbors']['All']=10

min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]



umapData = calculate_umap(CAll[CellIden],n_neighbors, min_dist)
if visualize:
  

  drawUMAP(umapData, ['H4'],CAll,settings,
              title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
              Figname = name+'1_UMAP_(core)')

# umap_params(k,CellIden,dir_plots,
#             valA = min_dist, valB = n_neighbors
#             )

# if j=='4':
#     ind  = np.where(umapData[:,0]<5)
#     k_orig.drop(k_orig.index[ind],inplace=True)
#     k = k_orig.copy();CAll=pd.concat([k]).copy()
    
#     umapData = calculate_umap(CAll[CellIden],n_neighbors, min_dist)
#     drawUMAP(umapData, ['H4'],CAll,settings,
#                 title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
#                 Figname = name+'1_UMAP_(core)')


### Dbscan and clusters

In [ ]:
s['min_samples']['1']=80
s['eps']['1']=0.4

s['min_samples']['2']=50
s['eps']['2']=0.2

s['min_samples']['4']=60
s['eps']['4']=0.15

s['min_samples']['5']=130
s['eps']['5']=0.17

s['min_samples']['All']=130
s['eps']['All']=0.17
eps=s['eps'][j];min_samples=s['min_samples'][j]
dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps,min_samples)
if visualize:
  drawDbscan(dbscanData,labels,core_samples_mask,settings,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname = name+'1_dbscan')
              
  


# dbscan_params(X_2d,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )


In [ ]:

plotClusters(k,umapData,labels,NamesAll,settings,
             title = 'Clusters Tumor '+j+' - Cell Iden Based',
             figname = name+'1_Clusters')
             



In [ ]:
print(groups)

### HeatMap

In [ ]:
# if j=='2' or j=='1':
#     K = k.copy() 
#     k = pd.concat([k.copy(),kCD45pos])
     
    

if visualize:
    for group_name,g in zip(groups[0],groups[1]):
        # cd45 added only on heat map
        group = g.copy()
        # group.append('CD45')
            
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'T'+j+' Cell Iden Based: '+group_name,
                figname = name+'1_HeatMap_'+group_name)
# if j=='2' or j=='1':
#     k=K
#     K = None


    # del K

### MeanDist

In [ ]:

if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title='MeanDist CellIdentity based: '+group_name,
                figname = name+'1_MeanDist_'+group_name)


## features on UMAP

In [ ]:

if visualize:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title ='Tumor'+j+ ' Cell Iden UMAP', 
            Figname = name+'2_UMAP_')

## save data to file

In [ ]:

dbscanData, umapData,k, CAll,labels,s = None,None,None,None,None,None


# Epigenetics Based

In [ ]:


k = k_orig.copy()
CAll=pd.concat([k]).copy()

s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}

name = 'Tumor'+j+'_Epigen_'



### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['1']=0.07
s['n_neighbors']['1']=30

s['min_dist']['2']=0.05
s['n_neighbors']['2']=30

s['min_dist']['4']=0.01
s['n_neighbors']['4']=150

s['min_dist']['5']=0.001
s['n_neighbors']['5']=10

s['min_dist']['All']=0.001
s['n_neighbors']['All']=10
min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]


umapData = calculate_umap(CAll[EpiCols],n_neighbors, min_dist)
if visualize:
    drawUMAP(umapData, ['H4'],CAll,settings,
              title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
              Figname = name+'1_UMAP_(core)')
    


# umap_params(k,MRK,dir_plots,
#             valA = min_dist, valB = n_neighbors,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.2, valB = 300,
#             )
# umap_params(k,MRK,dir_plots,
#             valA = 0.1, valB = 300,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.001, valB = 300,
#             )

### Dbscan and clusters

In [ ]:
   

s['min_samples']['1']=80
s['eps']['1']=0.4
s['min_samples']['2']=40
s['eps']['2']=0.2
s['min_samples']['4']=100
s['eps']['4']=0.15
s['min_samples']['5']=50
s['eps']['5']=0.1
s['min_samples']['All']=50
s['eps']['All']=0.1
min_samples = s['min_samples'][j]
eps = s['eps'][j]


dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps=eps,min_samples=min_samples)
if visualize:
  drawDbscan(dbscanData,labels,core_samples_mask,settings,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname = name+'1_dbscan')
              

In [ ]:
plotClusters(k,umapData,labels,NamesAll,settings,
             title = 'Clusters Tumor '+j+' - Epigen Based',
             figname = name+'1_Clusters')


## HeatMap

In [ ]:
# if j=='2' or j=='1':
#             K = k.copy() 
#             k = pd.concat([k.copy(),kCD45pos])
if visualize:
    for group_name,g in zip(groups[0],groups[1]):
        # cd45 added only on heat map
        group = g.copy()
        # group.append('CD45')
        
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'T'+j+' Epigen Based: '+group_name,
                figname = name+'1_HeatMap_'+group_name)
# if j=='2' or j=='1':
#     k=K
#     # del K
#     K = None

## Meandist

In [ ]:
if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title='MeanDist Epigen based: '+group_name,
                figname = name+'1_MeanDist_'+group_name)


## features on UMAP

In [ ]:

if visualize:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title ='Tumor '+j+' Epigen UMAP', 
            Figname = name+'2_UMAP_')
    

## save data to file

In [ ]:
dbscanData, umapData,k, CAll,labels,s = None,None,None,None,None,None

# ALL (Epigenetics + Cell identity Based)

In [ ]:

k = k_orig.copy()
CAll=pd.concat([k]).copy()

s ={}
s['min_dist'] = {}
s['n_neighbors'] = {}
s['min_samples'] = {}
s['eps'] = {}

name = 'Tumor'+j+'_CellIden+EpiCols_'



### UMAP

plot for each feature according to its intensity

In [ ]:
s['min_dist']['1']=0.07
s['n_neighbors']['1']=30

s['min_dist']['2']=0.05
s['n_neighbors']['2']=30

s['min_dist']['4']=0.01
s['n_neighbors']['4']=150

s['min_dist']['5']=0.001
s['n_neighbors']['5']=10

s['min_dist']['All']=0.001
s['n_neighbors']['All']=10

min_dist = s['min_dist'][j]
n_neighbors = s['n_neighbors'][j]


umapData = calculate_umap(CAll[CellIden_EpiCols],n_neighbors, min_dist)
if visualize:
    drawUMAP(umapData, ['H4'],CAll,settings,
              title = f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
              Figname = name+'1_UMAP_(core)')
    


# umap_params(k,MRK,dir_plots,
#             valA = min_dist, valB = n_neighbors,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.2, valB = 300,
#             )
# umap_params(k,MRK,dir_plots,
#             valA = 0.1, valB = 300,
#             )

# umap_params(k,MRK,dir_plots,
#             valA = 0.001, valB = 300,
#             )

### Dbscan and clusters

In [ ]:
   

s['min_samples']['1']=80
s['eps']['1']=0.4
s['min_samples']['2']=70
s['eps']['2']=0.17

s['min_samples']['4']=100
s['eps']['4']=0.17

s['min_samples']['5']=50
s['eps']['5']=0.1

s['min_samples']['All']=50
s['eps']['All']=0.1

min_samples = s['min_samples'][j]
eps = s['eps'][j]


dbscanData,labels,core_samples_mask = calculate_dbscan(umapData,eps=eps,min_samples=min_samples)
if visualize:
  drawDbscan(dbscanData,labels,core_samples_mask,settings,
              title=f'min_samples = {min_samples}, eps = {eps}',
              figname = name+'1_dbscan')
              
        
        

# dbscan_params(X_2d,dir_plots,
#                   valA = eps, valB = min_samples,
#                   itersA = 3, itersB = 3,
#                   )


  
 

In [ ]:
plotClusters(k,umapData,labels,NamesAll,settings,
             title = 'Clusters Tumor '+j+' - Epigen +Cell identity Based',
             figname = name+'1_Clusters')


## HeatMap

In [ ]:
# if j=='2' or j=='1':
#             K = k.copy() 
#             k = pd.concat([k.copy(),kCD45pos])
if visualize:
    for group_name,g  in zip(groups[0],groups[1]):
        # cd45 added only on heat map
        group = g.copy()
        # group.append('CD45')
           
        HeatMap(k[k.Clust!=-1],group,settings,
                title = 'T'+j+' Epigen+Cell Based: '+group_name,
                figname = name+'1_HeatMap_'+group_name)  
# if j=='2' or j=='1':
#     k=K
#     # del K
#     K = None

## Meandist

In [ ]:

if visualize:
    for group_name,group in zip(groups[0],groups[1]):
        MeanDist(k [k.Clust==0],k [k.Clust==1],group,settings,
                title='MeanDist Epigen+Cell based: '+group_name,
                figname = name+'1_MeanDist_'+group_name)


## features on UMAP

In [ ]:

if visualize:
    drawUMAP(umapData, NamesAll,CAll,settings,
            title ='Tumor '+j+' Epigen +Cell identity UMAP', 
            Figname = name+'2_UMAP_')
    

## save data to file

In [ ]:
dbscanData, umapData,k, CAll,labels,s = None,None,None,None,None,None

# copy to single pdf

In [ ]:
dict ={}
dict['k'] = k_orig

pickle_dump('k'+j+'_sample', dict,dir_data)
imList2pdf(dir_plots,j,['CellIden','EpiCols','CellIden+EpiCols'])

end = time.process_time()
print ('total run time =', end-start )